# Read event log

In [13]:
import pandas as pd
import mysql.connector

from IPython.display import display, Markdown

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def read_event_log():
    df = pd.read_sql("""
        select event_log.id, event_log.date, event_log.actor_id,
        user.username, event_log.event_id, event.name
        from event_log
        join event on event.id = event_log.event_id
        join user on user.id = event_log.actor_id
        order by event_log.id
    """, db)
    df.set_index("id", inplace=True)
    
    return delete_consecutive_events(df)

def delete_consecutive_events(df):
    df.sort_values(["actor_id", "date"], inplace=True)
    df["time_diff_seconds"] = df["date"].diff().map(lambda x: x.total_seconds())
    
    to_delete = (df["time_diff_seconds"] >= 0) & (df["time_diff_seconds"] < 3)
    df.drop(df[to_delete].index, inplace=True)
    
    return df

event_log = read_event_log()
event_log.tail(100)

,date,actor_id,username,event_id,name,time_diff_seconds
id,,,,,,
378277,2021-03-09 08:01:34,197401,Annika-Hemlein,5,entity/revision/add,8.0
378278,2021-03-09 08:01:41,197401,Annika-Hemlein,5,entity/revision/add,7.0
378281,2021-03-09 08:44:48,197401,Annika-Hemlein,5,entity/revision/add,2587.0
378287,2021-03-09 08:44:52,197401,Annika-Hemlein,5,entity/revision/add,4.0
378291,2021-03-09 08:45:00,197401,Annika-Hemlein,5,entity/revision/add,8.0
...,...,...,...,...,...,...
379200,2021-03-10 16:04:17,198070,Ryker,9,discussion/comment/create,-58325.0
379212,2021-03-10 20:34:46,198102,Vater,8,discussion/create,16229.0
379231,2021-03-11 08:11:54,198102,Vater,9,discussion/comment/create,41828.0


# Festlegung Event-Typen

```
+----+-----------------------------+-------------+
| id | name                        | description |
+----+-----------------------------+-------------+
|  1 | taxonomy/term/associate     | NULL        |
|  2 | taxonomy/term/create        | NULL        |
|  3 | license/object/set          | NULL        |
|  4 | entity/create               | NULL        |
|  5 | entity/revision/add         | NULL        |
|  6 | entity/revision/checkout    | NULL        |
|  7 | entity/link/create          | NULL        |
|  8 | discussion/create           | NULL        |
|  9 | discussion/comment/create   | NULL        |
| 10 | uuid/trash                  | NULL        |
| 11 | entity/revision/reject      | NULL        |
| 12 | taxonomy/term/update        | NULL        |
| 13 | uuid/restore                | NULL        |
| 14 | discussion/comment/archive  | NULL        |
| 15 | taxonomy/term/parent/change | NULL        |
| 16 | discussion/restore          | NULL        |
| 17 | taxonomy/term/dissociate    | NULL        |
| 18 | entity/link/remove          | NULL        |
+----+-----------------------------+-------------+
```

In [14]:
event_types = {
    "Inhaltsbearbeitungen": [5,6],
    "Kommentare": [8,9,14,16],
    "Reviewing": [6,11],
    "Taxonomy": [1,2,12,15,17],
    "Administratives": [10,13,3]
}

# Festlegung der zu betrachtenden Zeitintervalle

In [18]:
time_types = {
    "Quartal": 365.25/4, # Vierteljahr
    "Halbjahr": 365.25/2, # Halbjahr
    "Jahr": 365.25 # Jahr
}

In [94]:
import math
import numpy as np
import plotly.graph_objects as go

def plot_sankey(df, days_delta=90, event_ids=None,
                diff_start=None, basis=10):
    display(Markdown(f"Basis: {basis}"))
    
    if event_ids != None:
        display(Markdown(f"Event-Typen: {event_ids}"))
        df = df[df["event_id"].isin(event_ids)]
    
    if diff_start == None:
        diff_start = days_delta
    
    max_date = df["date"].max()
    
    display(Markdown(f"Vergleich Zeitraum vor {days_delta + diff_start} bis {diff_start} Tage zum Zeitraum vor {days_delta} Tage (heute = {max_date})."))
    
    def get_current(x):
        return x >= max_date - pd.Timedelta(days=days_delta) 
    
    def get_before(x):
        return ((x >= max_date - pd.Timedelta(days=days_delta + diff_start)) & 
                (x < max_date - pd.Timedelta(days=diff_start)))
    
    def count_current(d):
        return d[get_current(d)].count()

    def count_before(d):
        return d[get_before(d)].count()
    
    df = df[get_current(df["date"]) | get_before(df["date"])]
    df = df.groupby("username").aggregate({
        "date": [count_current, count_before]
    })
    df.columns = df.columns.droplevel(0)
    
    def edit_cat(edit_count):
        if edit_count == 0: return 0
        
        return 1+math.floor(math.log(edit_count) / math.log(basis))
    
    df["target"] = df["count_current"].map(edit_cat)
    df["source"] = df["count_before"].map(edit_cat)
    
    df = df.groupby(["source", "target"]).count()
    df.reset_index(inplace=True)
    
    def compute_value(x, ref="source"):
        return x["count_current"] / df[df[ref] == x[ref]]["count_current"].sum() * 100

    df["value"] = df["count_current"]
    df["value_source"] = df.apply(compute_value, axis=1)
    df["value_target"] = df.apply(lambda x: compute_value(x, "target"), axis=1)

    cats = max(df["target"].max(), df["source"].max())
    labels = [0] + [basis**i for i in range(cats)]
    labels = [f">= {x}" for x in labels]
    #labels = ["== 0"] + [f"<= {basis**i}" for i in range(cats)]
    
    def show(kind, explanation, suffix=""):
        sankey=go.Sankey(
            arrangement="snap",
            valueformat=".0f",
            valuesuffix=suffix,
            node = dict(
              line = dict(color = "black", width = 0.5),
              label = 2*labels,
              color = "#007ec1",
              x = [0 for i in range(len(labels))] + [1 for i in range(len(labels))],
              y = 2*[i/len(labels) for i in range(len(labels))],
            ),
            link = dict(
              source = df["source"],
              target = df["target"] + len(labels),
              value = df[kind]
          ))

        fig = go.Figure(data=[sankey])
        fig.update_layout(title_text=f"Sankey-Diagramm: {explanation}",
                          font_size=12)
        fig.show()
    
    show("value", "Absolute Zahlen", " Autor:innen")
    show("value_source", "Prozentzahlen vom Ausgangsknoten", suffix="%")
    show("value_target", "Prozentzahlen vom Endknoten", suffix="%")


plot_sankey(event_log, event_ids=[5])

Basis: 10

Event-Typen: [5]

Vergleich Zeitraum vor 180 bis 90 Tage zum Zeitraum vor 90 Tage (heute = 2021-03-11 22:53:52).

# Verschiedene Zeitbereiche mit dem jeweiligen Zeitbereich davor

In [98]:
for time_type, delay in time_types.items():
    for basis in [5,10,20]:
        display(Markdown(f"## Letztes {time_type} im Vergleich zum gleichen Zeitraum davor (alle Events zusammengenommen)"))
        plot_sankey(event_log, days_delta=delay, basis=basis, event_ids=[5])

## Letztes Quartal im Vergleich zum gleichen Zeitraum davor (alle Events zusammengenommen)

Basis: 5

Event-Typen: [5]

Vergleich Zeitraum vor 182.625 bis 91.3125 Tage zum Zeitraum vor 91.3125 Tage (heute = 2021-03-11 22:53:52).

## Letztes Quartal im Vergleich zum gleichen Zeitraum davor (alle Events zusammengenommen)

...

Vergleich Zeitraum vor 730.5 bis 365.25 Tage zum Zeitraum vor 365.25 Tage (heute = 2021-03-11 22:53:52).

## Letztes Jahr im Vergleich zum gleichen Zeitraum davor (alle Events zusammengenommen)

Basis: 20

Event-Typen: [5]

Vergleich Zeitraum vor 730.5 bis 365.25 Tage zum Zeitraum vor 365.25 Tage (heute = 2021-03-11 22:53:52).

In [96]:
for time_type, delay in time_types.items():
    display(Markdown(f"## Letztes {time_type} im Vergleich zum Vorjahr (alle Events zusammengenommen)"))
    for basis in [5,10,20]:
        plot_sankey(event_log, days_delta=delay, diff_start=365.25, basis=basis)

## Letztes Quartal im Vergleich zum Vorjahr (alle Events zusammengenommen)

Basis: 5

Vergleich Zeitraum vor 456.5625 bis 365.25 Tage zum Zeitraum vor 91.3125 Tage (heute = 2021-03-11 22:53:52).

Basis: 10

Vergleich Zeitraum vor 456.5625 bis 365.25 Tage zum Zeitraum vor 91.3125 Tage (heute = 2021-03-11 22:53:52).

...

Vergleich Zeitraum vor 730.5 bis 365.25 Tage zum Zeitraum vor 365.25 Tage (heute = 2021-03-11 22:53:52).

Basis: 10

Vergleich Zeitraum vor 730.5 bis 365.25 Tage zum Zeitraum vor 365.25 Tage (heute = 2021-03-11 22:53:52).

Basis: 20

Vergleich Zeitraum vor 730.5 bis 365.25 Tage zum Zeitraum vor 365.25 Tage (heute = 2021-03-11 22:53:52).